# Insult Classification

In this exercise, we would like to filter out insulting comments on a web forum. 

To train our models, we have a list of historic comments with a judgement wether they're insulting or not.

In [4]:
import pandas as pd
path_to_insults = "C://Users//Sky//Downloads//data//data//"
data = pd.read_csv(path_to_insults + 'train-utf8.csv')
data.head(2)

,Insult,Date,Comment
0,1,20120618192155Z,You fuck your dad.
1,0,20120528192215Z,i really don't understand your point. It seem...


In [5]:
print ("%d comments, of which %d insults (%d%%)" % \
    (len(data), data.Insult.sum(), 100 * data.Insult.mean()))

3947 comments, of which 1049 insults (26%)


### Looking for known bad words

One way to do this, is to load Google's bad word list and flag comments that contain one or more words.

- Load `google_badlist.txt` from `data/insults/`
- Add a column to `data` with a flag (0 or 1) if the comment contains a bad word
- Compute the accuracy of this method - does this look good?
- What would a naive classifier's score be (i.e., always predicting 0 or 1)?
- Also compute the precision, recall, F1 score and AUC score
- What is your verdict?

In [5]:
filename = path_to_insults + 'google_badlist.txt'

'C://Users//Sky//Downloads//data//data//google_badlist.txt'

In [7]:
# Your code here

## First method

In [10]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
# Load the bad words list
with open(filename, 'r') as file:
    bad_words = [line.strip() for line in file.readlines()]

# Function to flag comments with bad words
def flag_comment(comment):
    return any(bad_word in comment for bad_word in bad_words)

# Flag each comment
data['flag'] = data['Comment'].apply(flag_comment).astype(int)

# Compute accuracy if you have true labels
true_labels = data['Insult']
accuracy = accuracy_score(true_labels, data['flag'])
f1 = f1_score(true_labels, data['flag'])
auc = roc_auc_score(true_labels, data['flag'])

print(f'Accuracy: {accuracy}')
print(f'F1 Score: {f1}')
print(f'AUC Score: {auc}')

Accuracy: 0.6559412211806436
F1 Score: 0.4080209241499564
AUC Score: 0.5890116190713032


#### This method provides a naive implementation of insult recognition and produces a better accuracy than a random guess. This method is intuitive but not satisfactory in accuracy since it does not consider the meaning of the entire sentence, as it only evaluates whether the words in the list appears or not. Therefore, sometimes people use words in the list as an modal word instead of insult. 

### Learning bad words on the fly

Another way of doing this, is to learn the insulting words on the fly using `CountVectorizer`. 

Please refer to the scikit learn tutorial at 'http://scikit-learn.org/stable/tutorial/text_analytics/working_with_text_data.html' if you need some help.

Here is what you need to do:

- Import `CountVectorizer` from `sklearn.feature_extraction.text`
- Train the `CountVectorizer` on the insults and create a feature set $X$ representing words in the comments
- Train `MultinomialNB` and `BernoulliNB` from `scikitsklearn`  on the new feature set $X$
- Using cross-validation, compute the accuracy, precision, recall, F1 and AUC of your model
- What is your verdict?

NOTE: The F1 score is another useful score to compute when one of the two classes is very rare. We didn't go over it in class but it's basically the harmonic mean between precision and recall and goes from 0 (min) to 1 (max).  You can see more here: 'https://en.wikipedia.org/wiki/F1_score' 

## Multinomial and Bernoulli

In [28]:
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import cross_val_score
from sklearn.naive_bayes import BernoulliNB
import numpy as np

In [6]:
# Your code here

In [30]:
# Split the data into training and test sets
X_train = data['Comment']
y_train = data['Insult']

# Vectorize the comments
vectorizer = CountVectorizer(stop_words='english')
X_vec = vectorizer.fit_transform(X_train)

# Initialize the Naive Bayes classifier
modelMul = MultinomialNB()
modelBer = BernoulliNB()

# Train the model
modelMul.fit(X_vec, y_train)
modelBer.fit(X_vec, y_train)

# Calculate the metrics
accuracyMul =  cross_val_score(modelMul, X_vec, y_train, cv=5, scoring='accuracy')
accuracyBer =  cross_val_score(modelBer, X_vec, y_train, cv=5, scoring='accuracy')
precisionMul = cross_val_score(modelMul, X_vec, y_train, cv=5, scoring='precision')
precisionBer = cross_val_score(modelBer, X_vec, y_train, cv=5, scoring='precision')
recallMul = cross_val_score(modelMul, X_vec, y_train, cv=5, scoring='recall')
recallBer = cross_val_score(modelBer, X_vec, y_train, cv=5, scoring='recall')
f1Mul = cross_val_score(modelMul, X_vec, y_train, cv=5, scoring='f1')
f1Ber = cross_val_score(modelBer, X_vec, y_train, cv=5, scoring='f1')
aucMul = cross_val_score(modelMul, X_vec, y_train, cv=5, scoring='roc_auc')
aucBer = cross_val_score(modelBer, X_vec, y_train, cv=5, scoring='roc_auc')

# Print the metrics
print(f'Multinomial Accuracy: {np.mean(accuracyMul)}')
print(f'Multinomial Precision: {np.mean(precisionMul)}')
print(f'Multinomial Recall: {np.mean(recallMul)}')
print(f'Multinomial F1 Score: {np.mean(f1Mul)}')
print(f'Multinomial AUC Score: {np.mean(aucMul)}')

print(f'Bernoulli Accuracy: {np.mean(accuracyBer)}')
print(f'Bernoulli Precision: {np.mean(precisionBer)}')
print(f'Bernoulli Recall: {np.mean(recallBer)}')
print(f'Bernoulli F1 Score: {np.mean(f1Ber)}')
print(f'Bernoulli AUC Score: {np.mean(aucBer)}')


Multinomial Accuracy: 0.7539914328343842
Multinomial Precision: 0.5322186916711257
Multinomial Recall: 0.6148598769651402
Multinomial F1 Score: 0.5705229719971298
Multinomial AUC Score: 0.7838520003521507
Bernoulli Accuracy: 0.727891739262967
Bernoulli Precision: 0.46098864098864095
Bernoulli Recall: 0.13254044201412624
Bernoulli F1 Score: 0.2056149133884983
Bernoulli AUC Score: 0.7471143891382318


#### The performances in accuracy of Multinomial and Bernoulli are similar, with accuracies around 75%, which is an obvious improvement comparing to the naive classifier used above. The use of vectorizer takes every word in the training set into consideration and Multinomial and Bernoulli classifiers uses every word in a comment to evaluate whether the comment is an insult or not. This compensates the drawback I stated before, and including the context makes the prediction more accurate. However, the statistics tells that Multinomial outperforms Bernoulli since Bernoulli's precision is lower than Multinomial's, which indicates that of all instances predicted as positive by the Bernoulli classifier, a smaller proportion was actually positive. This suggests many false positives. Also, the recall of the Bernoulli classifier is quite low, which means it's missing a lot of true positive cases (high false negative rate). It only correctly identifies about 13.25% of the actual positive instances. Given that the F1 score is the harmonic mean of precision and recall, it is greatly affected by the low recall. Because recall is so low, even a moderate precision cannot bring the F1 score up significantly. Therefore, even though their accuracy are similar, Multinomial would perform better given a larger dataset with balanced data.